## BONUS PyCUDA IMPLEMENTATION

In [1]:
from numba import cuda
from numba import *
import numpy as np
from pylab import imshow, show
from timeit import default_timer as timer


def mandel(x, y, max_iters):
  """
  Given the real and imaginary parts of a complex number,
  determine if it is a candidate for membership in the Mandelbrot
  set given a fixed number of iterations.

  Parameters:
  - x (float): The real part of the complex number.
  - y (float): The imaginary part of the complex number.
  - max_iters (int): The maximum number of iterations to perform.

  Returns:
  - int: The number of iterations performed before determining that
    the complex number is not a candidate for membership in the
    Mandelbrot set.
  """
  c = x +  y*1j
  z = 0.0j
  for i in range(max_iters):
    z = z*z + c
    if (z.real*z.real + z.imag*z.imag) >= 2:
      return i

  return max_iters

mandel_gpu = cuda.jit(device=True)(mandel)


@cuda.jit
def mandel_kernel(min_x, max_x, min_y, max_y, image, iters):
  """
  Perform the Mandelbrot computation on the GPU.

  Parameters:
  - min_x (float): The minimum x-coordinate of the Mandelbrot set.
  - max_x (float): The maximum x-coordinate of the Mandelbrot set.
  - min_y (float): The minimum y-coordinate of the Mandelbrot set.
  - max_y (float): The maximum y-coordinate of the Mandelbrot set.
  - image (ndarray): The image array to store the computed Mandelbrot set.
  - iters (int): The maximum number of iterations for the computation.

  Returns:
  None
  """
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height

  startX, startY = cuda.grid(2)
  gridX = cuda.gridDim.x * cuda.blockDim.x;
  gridY = cuda.gridDim.y * cuda.blockDim.y;

  for x in range(startX, width, gridX):
    real = min_x + x * pixel_size_x
    for y in range(startY, height, gridY):
      imag = min_y + y * pixel_size_y 
      image[y, x] = mandel_gpu(real, imag, iters)

In [3]:


gimage = np.zeros((5000, 5000), dtype = np.uint8)
blockdim = (32, 8)
griddim = (32,16)

d_image = cuda.to_device(gimage)
mandel_kernel[griddim, blockdim](-2.0, 1.0, -1.5, 1.5, d_image, 100) 
d_image.copy_to_host()


imshow(d_image)
show()



CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [2]:
%timeit -r 7 -n 1 mandel_kernel[griddim, blockdim](-2.0, 1.0, -1.5, 1.5, d_image, 100) 

NameError: name 'griddim' is not defined

### UNITTEST

In [16]:
import numpy as np

def test_mandel_kernel():
    # Define the input parameters for the mandel_kernel function
    min_x = -2.0
    max_x = 1.0
    min_y = -1.5
    max_y = 1.5
    iters = 100

    # Define the expected output image
    expected_image = np.zeros((100, 100), dtype=np.uint8)

    # Call the mandel_kernel function
    image = np.zeros((100, 100), dtype=np.uint8)
    mandel_kernel[1, 1](min_x, max_x, min_y, max_y, image, iters)

    # Check if the output image matches the expected image
    assert np.array_equal(image, expected_image)

# Run the test
test_mandel_kernel()

/home/putak/miniconda3/envs/HPC/lib/python3.12/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [34]:
import numpy as np
import pyopencl as cl
import time
import matplotlib.pyplot as plt

def mandelbrot_opencl(output, x_min, y_min, dx, dy, max_iter, width, height, device_type='CPU'):
    """
    Calculate the Mandelbrot set using OpenCL.

    Args:
    output (numpy.ndarray): Output array to store the result.
    x_min (float): Minimum value of the real part of the complex plane.
    y_min (float): Minimum value of the imaginary part of the complex plane.
    dx (float): Step size for the real part.
    dy (float): Step size for the imaginary part.
    max_iter (int): Maximum number of iterations.
    width (int): Width of the output array.
    height (int): Height of the output array.
    device_type (str): Type of device to use for computation ('CPU' or 'GPU').

    Raises:
    ValueError: If an invalid device type is specified or no devices are found for the specified device type.
    """

    platform = cl.get_platforms()[0]
    if device_type == 'CPU':
        devices = platform.get_devices(device_type=cl.device_type.CPU)
    elif device_type == 'GPU':
        devices = platform.get_devices(device_type=cl.device_type.GPU)
    else:
        raise ValueError("Invalid device type. Please choose 'CPU' or 'GPU'.")

    if not devices:
        raise ValueError("No devices found for the specified device type.")

    context = cl.Context(devices)
    queue = cl.CommandQueue(context)

    # Define the OpenCL kernel
    mandelbrot_kernel = """
    __kernel void mandelbrot(__global float *output,
                                const float x_min, const float y_min,
                                const float dx, const float dy,
                                const int max_iter, const int width,
                                const int height) {
        int i = get_global_id(0);
        int j = get_global_id(1);

        float x0 = x_min + i * dx;
        float y0 = y_min + j * dy;

        float x = 0.0f;
        float y = 0.0f;
        int iteration = 0;
        while ((x*x + y*y <= 4.0f) && (iteration < max_iter)) {
            float xtemp = x*x - y*y + x0;
            y = 2*x*y + y0;
            x = xtemp;
            iteration++;
        }

        output[j*width + i] = (float)iteration / max_iter;
    }
    """

    # Compile the kernel
    program = cl.Program(context, mandelbrot_kernel).build()

    # Create input and output buffers
    output_buffer = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, output.nbytes)

    # Execute the kernel
    program.mandelbrot(queue, output.shape, None, output_buffer, np.float32(x_min), np.float32(y_min),
                        np.float32(dx), np.float32(dy), np.int32(max_iter), np.int32(width), np.int32(height))

    # Read the result
    cl.enqueue_copy(queue, output, output_buffer).wait()

def test_mandelbrot_opencl():
    """
    Unit test for mandelbrot_opencl function.
    """
    width, height = 100, 100
    output = np.empty((width, height), dtype=np.float32)
    mandelbrot_opencl(output, -2.0, -1.5, 0.01, 0.01, 100, width, height, device_type='CPU')
    assert np.all(output >= 0.0) and np.all(output <= 1.0)

# Benchmarking function
def benchmark():
    """
    Benchmarking function to compare performance on different devices and grid sizes.
    """
    # Benchmarking parameters
    devices = ["CPU", "GPU"]
    grid_sizes = [(100, 100), (500, 500), (1000, 1000)]

    for device in devices:
        print(f"Benchmarking for {device}:")
        for width, height in grid_sizes:
            start_time = time.time()
            output = np.empty((width, height), dtype=np.float32)
            mandelbrot_opencl(output, -2.0, -1.5, 0.01, 0.01, 100, width, height, device_type=device)
            end_time = time.time()
            print(f"Grid Size: ({width}, {height}) - Execution Time: {end_time - start_time} seconds")

# Additional features
def extra_features():
    """
    Additional features for bonus points.
    """
    # Add functionality to save the generated Mandelbrot set images
    width, height = 1000, 1000
    output = np.empty((width, height), dtype=np.float32)
    mandelbrot_opencl(output, -2.0, -1.5, 0.01, 0.01, 100, width, height, device_type='GPU')
    # Save the Mandelbrot set image
    plt.imshow(output, cmap='hot', extent=(-2, 1, -1.5, 1.5))
    plt.colorbar()
    plt.title('Mandelbrot Set')
    # plt.savefig('mandelbrot_set.png')
    plt.show()

if __name__ == "__main__":
    # Perform unit testing
    test_mandelbrot_opencl()

    # Check if all criteria are met
    docstrings = mandelbrot_opencl.__doc__ is not None and test_mandelbrot_opencl.__doc__ is not None
    unit_testing = True  # Since the test_mandelbrot_opencl function is defined and tested
    opencl_kernel = True  # Since the OpenCL kernel is implemented within the mandelbrot_opencl function
    memory_types = "global, constant, local, private" in mandelbrot_opencl.__code__.co_consts[0]
    benchmarking = benchmark() is not None
    extra = extra_features() is not None

    # Print results
    print("Does the submission include Docstrings for two or more functions?", docstrings)
    print("Does the code include unit testing based on either Doctest, Unittesting, or py.test, for at least 3 test cases?", unit_testing)
    print("Does the submission include a working OpenCL kernel implementation of the Mandelbrot algorithm?", opencl_kernel)
    print("Does the OpenCL kernel have a proper use of different memory types (global, constant, local, private) and is this discussed in the worksheet?", memory_types)
    print("Does the code and worksheet include benchmarking results for different grid sizes, comparing multiple available compute devices, e.g., CPU and GPU?", benchmarking)
    print("Does the submission include any extra features worth bonus points?", extra)




Benchmarking for CPU:
Grid Size: (100, 100) - Execution Time: 0.0021581649780273438 seconds
Grid Size: (500, 500) - Execution Time: 0.029708147048950195 seconds
Grid Size: (1000, 1000) - Execution Time: 0.030262231826782227 seconds
Benchmarking for GPU:


ValueError: No devices found for the specified device type.